In [1]:
%load_ext autoreload
%autoreload 2
from data.dataloader import get_tokenizer
from model.model import GPT
from data.utils import Trie
import pickle
import numpy as np 
import torch

from data.dataloader import SearchData
from torch.utils.data import DataLoader
from model.env import Env
from torch.utils.tensorboard import SummaryWriter

from train_rl import train_q_learning_iterative, train_policy_gradient, train_policy_gradient_iterative


In [2]:
tokenizer = get_tokenizer()
actor = GPT(tokenizer=tokenizer, vocab_size=len(tokenizer), block_size=100).cuda()
reader = GPT(tokenizer=tokenizer, vocab_size=len(tokenizer), block_size=100).cuda()
reader_pt = torch.load('checkpoints/baseline/gt_0.001/baseline_23.pt')
reader.load_state_dict(reader_pt)
optimizer = torch.optim.Adam(actor.parameters(), lr=1e-3)


In [3]:
with open('data/trie.pkl', 'rb') as f:
    trie = pickle.load(f)
with open('data/dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)
    _, database = dataset

In [4]:
dataset = SearchData('data/train.pkl', tokenizer)
loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

In [5]:
env = Env(reader, database, trie, tokenizer, max_len=100).cuda()

In [8]:
writer = SummaryWriter('runs/debugv2')

In [9]:
train_policy_gradient(actor, optimizer, loader, env, writer=writer, reward_type='ce', gamma=0.9)

KeyboardInterrupt: 

In [13]:
from train_rl import train_policy_gradient_iterative
actor = GPT(tokenizer=tokenizer, vocab_size=len(tokenizer), block_size=100).cuda()
reader = GPT(tokenizer=tokenizer, vocab_size=len(tokenizer), block_size=100).cuda()
optimizer = torch.optim.Adam(actor.parameters(), lr=1e-3)
env = Env(reader, database, trie, tokenizer, max_len=100).cuda()
env_opt = torch.optim.Adam(reader.parameters(), lr=1e-3)

writer = SummaryWriter('runs/pg_acc_iterative')
train_policy_gradient_iterative(actor, optimizer, env_opt, loader, env, writer=writer, reward_type='acc', gamma=0.9)

Episode 2500: reward=0.0938, policy_loss=-0.0001, accuracy=0.0938: 100%|██████████| 2500/2500 [05:05<00:00,  8.18it/s]


In [17]:

actor = GPT(tokenizer=tokenizer, vocab_size=len(tokenizer), block_size=100).cuda()
reader = GPT(tokenizer=tokenizer, vocab_size=len(tokenizer), block_size=100).cuda()
optimizer = torch.optim.Adam(actor.parameters(), lr=1e-3)
env = Env(reader, database, trie, tokenizer, max_len=100).cuda()
env_opt = torch.optim.Adam(reader.parameters(), lr=1e-3)

writer = SummaryWriter('runs/q_acc_iterative')
train_q_learning_iterative(actor, optimizer, env_opt, loader, env, writer=writer, reward_type='acc', gamma=0.9)

KeyboardInterrupt: 

In [25]:

actor = GPT(tokenizer=tokenizer, vocab_size=len(tokenizer), block_size=100).cuda()
reader = GPT(tokenizer=tokenizer, vocab_size=len(tokenizer), block_size=100).cuda()
optimizer = torch.optim.Adam(actor.parameters(), lr=1e-3)
env = Env(reader, database, trie, tokenizer, max_len=100).cuda()
env_opt = torch.optim.Adam(reader.parameters(), lr=1e-3)

writer = SummaryWriter('runs/q_ce_iterative_eps_greedy')
train_q_learning_iterative(actor, optimizer, env_opt, loader, env, writer=writer, reward_type='ce', gamma=0.99)







/home/wogns98/CS492_search_engine/train_rl.py:171: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(q_values, target_q_values, reduction='none') * (1.-terminates.float().view(-1, 1))
























































































































































































































































































































































































































